In [1]:
import pandas as pd

### Load the data

In [2]:
female = pd.read_csv('./data/female.csv')
male = pd.read_csv('./data/male.csv')

print("Female dataset shape:", female.shape)
print("Male dataset shape:", male.shape)

Female dataset shape: (1986, 108)
Male dataset shape: (4082, 108)


### Define size percentile

In [3]:
percentile_ranges = [(0,5),(5,25),(25,50),(50,75),(75,90),(90,97),(97,100)]
sizes = ['XS', 'S', 'M', 'L', 'XL', 'XXL', 'XXXL']

print("Size → Percentile mapping:")
for size, pr in zip(sizes, percentile_ranges):
    print(size, ":", pr)

Size → Percentile mapping:
XS : (0, 5)
S : (5, 25)
M : (25, 50)
L : (50, 75)
XL : (75, 90)
XXL : (90, 97)
XXXL : (97, 100)


### Convert percentiles to actual numbers

In [4]:
female = pd.read_csv('./data/female.csv')
male = pd.read_csv('./data/male.csv')
for low, high in percentile_ranges:
    chest_low = female['chestcircumference'].quantile(low / 100)
    chest_high = female['chestcircumference'].quantile(high / 100)

    print(f"Chest {low}-{high}% : {chest_low:.1f} to {chest_high:.1f}")

for low, high in percentile_ranges:
    chest_low_male = male['chestcircumference'].quantile(low / 100)
    chest_high_female = male['chestcircumference'].quantile(high / 100)

    print(f"Chest {low}-{high}% : {chest_low_male:.1f} to {chest_high_female:.1f}")


Chest 0-5% : 695.0 to 824.2
Chest 5-25% : 824.2 to 889.0
Chest 25-50% : 889.0 to 940.0
Chest 50-75% : 940.0 to 999.0
Chest 75-90% : 999.0 to 1057.0
Chest 90-97% : 1057.0 to 1117.5
Chest 97-100% : 1117.5 to 1266.0
Chest 0-5% : 774.0 to 922.0
Chest 5-25% : 922.0 to 996.0
Chest 25-50% : 996.0 to 1056.0
Chest 50-75% : 1056.0 to 1117.0
Chest 75-90% : 1117.0 to 1172.0
Chest 90-97% : 1172.0 to 1233.0
Chest 97-100% : 1233.0 to 1469.0


### Size chart

In [5]:
def build_size_chart(data):
    chart = {}

    for size, (low, high) in zip(sizes, percentile_ranges):
        chart[size] = {
            'chest_min': data['chestcircumference'].quantile(low / 100),
            'chest_max': data['chestcircumference'].quantile(high / 100),
            'shoulder_min': data['biacromialbreadth'].quantile(low / 100),
            'shoulder_max': data['biacromialbreadth'].quantile(high / 100),
        }

    return chart

female_chart = build_size_chart(female)
male_chart = build_size_chart(male)

print("\nFemale Size Chart:")
for size, values in female_chart.items():
    print(size, values)

print("\nMale Size Chart:")
for size, values in male_chart.items():
    print(size, values)



Female Size Chart:
XS {'chest_min': 695.0, 'chest_max': 824.25, 'shoulder_min': 283.0, 'shoulder_max': 335.0}
S {'chest_min': 824.25, 'chest_max': 889.0, 'shoulder_min': 335.0, 'shoulder_max': 353.0}
M {'chest_min': 889.0, 'chest_max': 940.0, 'shoulder_min': 353.0, 'shoulder_max': 365.0}
L {'chest_min': 940.0, 'chest_max': 999.0, 'shoulder_min': 365.0, 'shoulder_max': 378.0}
XL {'chest_min': 999.0, 'chest_max': 1057.0, 'shoulder_min': 378.0, 'shoulder_max': 389.0}
XXL {'chest_min': 1057.0, 'chest_max': 1117.45, 'shoulder_min': 389.0, 'shoulder_max': 400.0}
XXXL {'chest_min': 1117.45, 'chest_max': 1266.0, 'shoulder_min': 400.0, 'shoulder_max': 422.0}

Male Size Chart:
XS {'chest_min': 774.0, 'chest_max': 922.0, 'shoulder_min': 337.0, 'shoulder_max': 384.0}
S {'chest_min': 922.0, 'chest_max': 996.0, 'shoulder_min': 384.0, 'shoulder_max': 403.0}
M {'chest_min': 996.0, 'chest_max': 1056.0, 'shoulder_min': 403.0, 'shoulder_max': 415.0}
L {'chest_min': 1056.0, 'chest_max': 1117.0, 'shoulder

### Assign to single person

In [6]:
def get_size(value, min_key, max_key, chart):
    for size, limits in chart.items():
        if limits[min_key] <= value < limits[max_key]:
            return size
    return None


# Test on ONE person
person = female.iloc[0]
person1 = male.iloc[0]

print("\nTesting first female person:")
print("Chest:", person['chestcircumference'])
print("Shoulder:", person['biacromialbreadth'])



print("Chest size:",
      get_size(person['chestcircumference'], 'chest_min', 'chest_max', female_chart))

print("Shoulder size:",
      get_size(person['biacromialbreadth'], 'shoulder_min', 'shoulder_max', female_chart))

print("\nTesting first male person:")
print("Chest:", person1['chestcircumference'])
print("Shoulder:", person1['biacromialbreadth'])

print("Chest size:",
      get_size(person['chestcircumference'], 'chest_min', 'chest_max', male_chart))

print("Shoulder size:",
      get_size(person['biacromialbreadth'], 'shoulder_min', 'shoulder_max', male_chart))



Testing first female person:
Chest: 922
Shoulder: 373
Chest size: M
Shoulder size: L

Testing first male person:
Chest: 1074
Shoulder: 401
Chest size: S
Shoulder size: XS


### Count Matches

In [7]:
matches = 0
conflicts = 0

for _, row in female.iterrows():
    chest_size = get_size(row['chestcircumference'], 'chest_min', 'chest_max', female_chart)
    shoulder_size = get_size(row['biacromialbreadth'], 'shoulder_min', 'shoulder_max', female_chart)

    if chest_size == shoulder_size:
        matches += 1
    else:
        conflicts += 1

print("\nFinal result for women:")
print("Matches:", matches)
print("Conflicts:", conflicts)

matches1 = 0
conflicts1 = 0

for _, row in male.iterrows():
    chest_size = get_size(row['chestcircumference'], 'chest_min', 'chest_max', male_chart)
    shoulder_size = get_size(row['biacromialbreadth'], 'shoulder_min', 'shoulder_max', male_chart)

    if chest_size == shoulder_size:
        matches1 += 1
    else:
        conflicts1 += 1

print("\nFinal result for men:")
print("Matches:", matches1)
print("Conflicts:", conflicts1)





Final result for women:
Matches: 469
Conflicts: 1517

Final result for men:
Matches: 1180
Conflicts: 2902
